In [1]:
import csv
import requests
import json
import datetime
import pandas as pd

Reading in Bitcoin to USD rates over time. Csv source is from https://www.coindesk.com/price/

In [2]:
with open('coindesk-bpi-USD-close_data-2013-08-23_2018-01-08.csv') as csvfile:
    readCSV = csv.reader(csvfile, delimiter=',')
    next(readCSV, None) #skip header
    dates = []
    prices = []
    for row in readCSV:
        date = row[0]
        price = row[1]

        dates.append(date)
        prices.append(price)

In [3]:
btc_usd_pd = pd.DataFrame(
    {'date': dates,
     'BTC_USD': prices,
    })
btc_usd_pd.head()

,BTC_USD,date
0,107.55,2013-08-23 00:00:00
1,108.69,2013-08-24 00:00:00
2,111.79,2013-08-25 00:00:00
3,112.23,2013-08-26 00:00:00
4,117.45,2013-08-27 00:00:00


For alternative currencies I use [Poloniex's api](https://poloniex.com/support/api/). Specifically, I use `returnChartData` from the Public API Method.

Reading in Litecoin to Bitcoin rate over time 

In [4]:
bl = requests.get('https://poloniex.com/public?command=returnChartData&currencyPair=BTC_LTC&start=1377302400&end=9999999999&period=86400')
btc_ltc_d = bl.json()

In [5]:
temp = [i['date'] for i in btc_ltc_d]
btc_ltc_dates = []
for date in temp:
    btc_ltc_dates.append(datetime.datetime.fromtimestamp(date).strftime('%Y-%m-%d 00:00:00'))

btc_ltc_prices = [float(i['close']) for i in btc_ltc_d]

In [6]:
btc_ltc_pd = pd.DataFrame(
    {'date': btc_ltc_dates,
     'BTC_LTC': btc_ltc_prices
    })
btc_ltc_pd.head()

,BTC_LTC,date
0,0.0285,2014-01-17 00:00:00
1,0.0283,2014-01-18 00:00:00
2,0.0283,2014-01-19 00:00:00
3,0.0283,2014-01-20 00:00:00
4,0.0283,2014-01-21 00:00:00


Reading in Ethereum to Bitcoin rate over time

In [7]:
be = requests.get('https://poloniex.com/public?command=returnChartData&currencyPair=BTC_ETH&start=1377302400&end=9999999999&period=86400')
btc_eth_d = be.json()

In [8]:
temp = [i['date'] for i in btc_eth_d]
btc_eth_dates = []
for date in temp:
    btc_eth_dates.append(datetime.datetime.fromtimestamp(date).strftime('%Y-%m-%d 00:00:00'))

btc_eth_prices = [float(i['close']) for i in btc_eth_d]

In [9]:
btc_eth_pd = pd.DataFrame(
    {'date': btc_eth_dates,
     'BTC_ETH': btc_eth_prices
    })
btc_eth_pd.head()

,BTC_ETH,date
0,0.003125,2015-08-07 00:00:00
1,0.002581,2015-08-08 00:00:00
2,0.002645,2015-08-09 00:00:00
3,0.003950,2015-08-10 00:00:00
4,0.004500,2015-08-11 00:00:00


Reading in Ripple to Bitcoin rate over time

In [10]:
br = requests.get('https://poloniex.com/public?command=returnChartData&currencyPair=BTC_XRP&start=1377302400&end=9999999999&period=86400')
btc_xrp_d = br.json()

In [11]:
temp = [i['date'] for i in btc_xrp_d]
btc_xrp_dates = []
for date in temp:
    btc_xrp_dates.append(datetime.datetime.fromtimestamp(date).strftime('%Y-%m-%d 00:00:00'))

btc_xrp_prices = [float(i['close']) for i in btc_xrp_d]

In [12]:
btc_xrp_pd = pd.DataFrame(
    {'date': btc_xrp_dates,
     'BTC_XRP': btc_xrp_prices
    })
btc_xrp_pd.head()

,BTC_XRP,date
0,0.000007,2014-08-13 00:00:00
1,0.000011,2014-08-14 00:00:00
2,0.000011,2014-08-15 00:00:00
3,0.000017,2014-08-16 00:00:00
4,0.000010,2014-08-17 00:00:00


Merging dataframes

In [13]:
m = pd.merge(btc_usd_pd, btc_ltc_pd, on="date",how="left")
m['BTC_USD'] = pd.to_numeric(m['BTC_USD'])
m['LTC_USD'] = m['BTC_USD']*m['BTC_LTC']

In [14]:
m1 = pd.merge(m,btc_eth_pd,on="date",how="left")
m1['ETH_USD'] = m1['BTC_USD']*m1['BTC_ETH']

In [15]:
m2 = pd.merge(m1,btc_xrp_pd,on="date",how="left")
m2['XRP_USD'] = m2['BTC_USD']*m2['BTC_XRP']

In [16]:
m2.loc[1589:1599]


,BTC_USD,date,BTC_LTC,LTC_USD,BTC_ETH,ETH_USD,BTC_XRP,XRP_USD
1589,14427.87,2017-12-29 00:00:00,0.016725,241.306270,0.055030,793.965686,0.000148,2.141962
1590,12629.81,2017-12-30 00:00:00,0.016380,206.877172,0.053493,675.606300,0.000143,1.809220
1591,13860.14,2017-12-31 00:00:00,0.016660,230.912704,0.056167,778.476385,0.000152,2.106741
1592,13412.44,2018-01-01 00:00:00,0.016975,227.681936,0.058369,782.874063,0.000149,1.995905
1593,14740.76,2018-01-02 00:00:00,0.016159,238.195646,0.062450,920.566948,0.000181,2.661149
1594,15134.65,2018-01-03 00:00:00,0.015590,235.949193,0.063550,961.808067,0.000184,2.777662
1595,15155.23,2018-01-04 00:00:00,0.014370,217.780655,0.057000,863.848110,0.000148,2.236912
1596,16937.17,2018-01-05 00:00:00,0.016200,274.385880,0.058500,990.824614,0.000154,2.606800
1597,17135.84,2018-01-06 00:00:00,0.016725,286.596924,0.069000,1182.372960,0.000170,2.918405
1598,16178.49,2018-01-07 00:00:00,0.016900,273.419717,0.075600,1223.093844,0.000161,2.596809
